# Implementation of [arXiv:1905.13729](https://arxiv.org/abs/1905.13729)
See also: DOI: 10.1103/PhysRevD.101.095032
## General solution to the U(1) anomaly equations

Given two set of  integers with equal dimension or the second one with an additional dimension, find a solution with zero sum and zero cubic-sum.

https://stackoverflow.com/a/43793179/2268280

In [2]:
import pandas as pd
import numpy as np
from astropy.table import Table
import itertools
import sys

def get_conditions(uf,nu_R,l=0,Dim=5):
    #uf=np.sort(np.unique(xdf))
    Ls=-l    
    Type='X'
    if Ls==0:
        Type='D'
        
    if Dim==5:
        d=2
    elif Dim==6:
        d=4
    
    md=[ {'{}→nu_R+{}fi'.format(Type,d):[nu_R,p]} for p in uf if nu_R+d*p-Ls==0 ]
    prmts=get_permutations(uf,3)
    if Dim==5:
        md=md+[ {'{}→nu_R+fi+fj'.format(  Type):list(p)} for p in prmts if nu_R in p and   p.sum()     -Ls==0 ]
    elif Dim==6:
        md=md+[ {'{}→nu_R+2fi+2fj'.format(Type):list(p)} for p in prmts if nu_R in p and 2*p.sum()-nu_R-Ls==0 ]        
    if md:
        return md
    else:
        return []
    
def get_permutations(l,n=3):
    prmts=[]
    for p in itertools. permutations(l, n):
        if sorted(p) not in prmts:
            prmts.append(sorted(p))
    return [ np.asarray(p) for p in prmts if isinstance(p,list) ]

def repeated(f):
    r={}
    r[1]=f
    for ri in range(2,f.size+2):
        r[ri]=r[ri-1][pd.Series(r[ri-1]).duplicated()]
        if ri>2:
            for a in np.unique(r[ri]):
                r[ri-1]=r[ri-1][ r[ri-1]!=a ]
                if len(r[ri-1])>0:
                    r[ri-1][ r[ri-1]!=a ]
        if len(r[ri])<=1:
            r[ri]=r[ri]
            r.pop(1)
            break
    return dict( (k,list(v)) for k,v in r.items() if len(v)!=0 )

def anomalies(row):
    f=np.array( [  row['f{}'.format(i)] for i in range(1,10) if row['f{}'.format(i)]!=0    ]  )
    rptd=repeated(f)
    # it triplets → remove triplet and check subanomaly cancelation:  Σsubf', 'Σsubf³' 
    # check four conditions
    return {'Σf':f.sum(), 'Σf³':(f**3).sum(),'multiplets':rptd}

#row=df.loc[28]
def get_solutions(row,solution='',Dim=5):
    if not solution:
        f=np.array( [  row['f{}'.format(i)] for i in range(1,10) if row['f{}'.format(i)]!=0    ]  )
    else:
        f=np.array(row[solution])
    rptd=repeated(f)

    nu_Rs=[]
    for i in rptd.keys():
        nu_Rs=nu_Rs+rptd.get(i)
    
    #if not rptd use all unique charges
    uf=np.sort(np.unique(f))    

    if not nu_Rs:
        nu_Rs=uf
        
    XD=[]
    for nu_R in nu_Rs:
        Ds=get_conditions(uf,nu_R,l=0,Dim=Dim)
        for D in Ds:
            if D not in XD:
                XD.append(D)
        #If nu_R in triplet → drop it and check the rest for X solutions
        if ( (rptd.get(3) and nu_R in rptd.get(3)) or
             (rptd.get(4) and nu_R in rptd.get(4)) ):
            l=nu_R # nu_R → -L
            xf=uf[uf!=l] #Drop nu_R
            # Drop l from nu_Rs and search new_nu_R there
            new_nu_Rs=nu_Rs.copy()
            new_nu_Rs.remove(l)
            for new_nu_R in new_nu_Rs:
                Xs=get_conditions(xf,new_nu_R,l,Dim=Dim)
                #print(l,new_nu_R,Xs)            
                for X in Xs:
                    if X not in XD:
                        XD.append(X)
        #TODO if rptd.get(4)
    return XD

def extract_multiplets(df,i,greater_than=0,column='solution'):
    dff=df[df[column].apply(
             lambda f: repeated(np.asarray(f))).str[i].apply( #get charges in multiplet i
             lambda l: len(l) if isinstance(l,list) else 0)>greater_than  # count charges in multiplets and filter for >
           ].reset_index(drop=True)
    return dff

def label_solutions(dsd,Dim=5):
    ds=dsd.copy()
    if Dim==5:
        dd=''
        dm=1
    elif Dim==6:
        dd='2'
        dm=2
    ds['nu_R']=ds.apply(lambda row: get_solutions(row,solution='solution',Dim=Dim)
                        ,axis='columns')
    ds['DarkDirac']=ds['nu_R'].apply(lambda l: [ 'D→nu_R+{}fi+{}fj'.format(dd,dd) in d.keys() for d in l] 
                         ).apply(lambda l: [True for T in l if T] ).apply(len)
    ds['DarkMajor']=ds['nu_R'].apply(lambda l: [ 'D→nu_R+{}fi'.format(2*dm) in d.keys() for d in l] 
                         ).apply(lambda l: [True for T in l if T] ).apply(len)
    ds['XDirac']=ds['nu_R'].apply(lambda l: [ 'X→nu_R+{}fi+{}fj'.format(dd,dd) in d.keys() for d in l] 
                         ).apply(lambda l: [True for T in l if T] ).apply(len)
    ds['XMajor']=ds['nu_R'].apply(lambda l: [ 'X→nu_R+{}fi'.format(2*dm) in d.keys() for d in l] 
                         ).apply(lambda l: [True for T in l if T] ).apply(len)
    return ds
def filter_solution(dsd,nmax=32):
    tmp=dsd.copy()
    tmp['sltn']=tmp['solution'].apply(lambda s: repeated(np.asarray(s)))
    cl=tmp[( ( (tmp['solution'].apply(np.asarray).apply(np.abs).apply(np.max)<=nmax) & 
              (tmp['sltn'].apply(lambda d:list(d.keys())).apply(len)>0) 
             ) &
         ((tmp['DarkDirac']>0) | (tmp['DarkMajor']>0) | (tmp['XDirac']>0) | (tmp['XMajor']>0))
       )].reset_index(drop=True)
    cl['-n']=-cl['n']

    cl=cl.sort_values(['-n','DarkDirac','DarkMajor','XDirac','XMajor'],ascending=False
            ).reset_index(drop=True)
    cl=cl.drop('-n',axis='columns')
    return cl

def get_condition(nuR,rls,label):
    CONDITION=False
    if label=='D→nu_R+fi+fj':
        if len(rls)==2 and nuR:
            if sum( nuR+rls )==0:
                CONDITION=True
    if label=='D→nu_R+2fi+2fj':
        if len(rls)==2 and nuR:
            if sum( nuR+[2*r for r in rls] )==0:
                CONDITION=True
    if label=='D→nu_R+2fi':
        if len(rls)==1 and nuR:
            if sum( nuR+[2*r for r in rls] )==0:
                CONDITION=True
    if label=='D→nu_R+4fi':
        if len(rls)==1 and nuR:
            if sum( nuR+[4*r for r in rls] )==0:
                CONDITION=True
    #TODO → IMPLEMENT SOLUTIONS FOR X:            
    if label=='X→nu_R+fi+fj':
        CONDITION=True
    if label=='X→nu_R+2fi+2fj':
        CONDITION=True
    if label=='X→nu_R+2fi':
        CONDITION=True
    if label=='X→nu_R+4fi':
        CONDITION=True
        
    return CONDITION 

def get_nuR_i(nrow,i,model,label):
    """
    Get the nu_R member of the solution
    """
    if nrow[model]>0 and nrow['sltn'].get(i) and nrow['nu_R'] and len( [ d.get(label) for d in nrow['nu_R'] if d.get(label)] )>0:
        M=[m  for m in  nrow['nu_R'] if m.get(label)]
        nuR=[n for n in nrow['sltn'][i] for m in M  if n in m.get(label)  ]
        nuR=list(np.unique(nuR))
        NUR=[]
        for i in range(len(nuR)):
            nu_R=nuR[i:i+1]
            try:
                #Get the proper solution
                sltn=[d.get(label) for d in M if nu_R in d.get(label)][0]
                rls=[rl for rl in sltn if rl not in nu_R]
            except:
                rls=[]
            condition=get_condition(nu_R,rls,label)
            if condition:
                NUR=NUR+nu_R
        
        if NUR:
            return NUR
        else:
            return []
    else:
        return []
    

def get_nuR(row,model='DarkDirac',Dim=5):
    '''
    Get nu_R for a model
    The main function is get_nbu
    '''
    nrow=row.copy() 
    if Dim==5:
        dd=''
        dm=1
    elif Dim==6:
        dd='2'
        dm=2    
    if model=='DarkDirac':
        label='D→nu_R+{}fi+{}fj'.format(dd,dd)
    elif model=='DarkMajor':
        label='D→nu_R+{}fi'.format(2*dm)
    elif model=='XDirac':
        label='X→nu_R+{}fi+{}fj'.format(dd,dd)
    else:
        label='X→nu_R+{}fi'.format(2*dm)
        
    if model=='XDirac' or model=='XMajor':
        if nrow[model]>0:
            if nrow['sltn'].get(3)  and len( nrow['sltn'].get(3)  ) <= 1:
                nrow['sltn']=dict( ((k,v) for k,v in nrow['sltn'].items() if k!=3) )
                i=2
                nuR=get_nuR_i(nrow,i,model,label)
            elif nrow['sltn'].get(3)  and len( nrow['sltn'].get(3)  ) == 2:
                nu_R=[]
                x,y=nrow['sltn'].get(3)
                l=[ d.get(label) for d in nrow['nu_R'] if label in d.keys()  ]
                l=list(itertools.chain(*l))
                if x in l: #remove y
                     nu_R.append(x)
                if y in l: #remove y
                     nu_R.append(y)
                return nu_R        
                #print('WARNING: two triplets in X solution not yet implemented in X solutions: {}'.format(nrow['solution']))
            elif nrow['sltn'].get(3)  and len( nrow['sltn'].get(3)  ) > 2:
                #Remove not nu_R
                #Check nu_R in solution
                print( dict( ((k,v) for k,v in nrow['sltn'].items() if k!=3) ) )
                print('WARNING: More than one triplet in X solution not yet implemented in X solutions: {}'.format(nrow['solution']))
        
    # model=='DDirac' or model=='DMajor'
    else:
        i=3
        nuR=get_nuR_i(nrow,i,model,label)
        if nuR:
            return nuR
        else:
            i=4
            nuR=get_nuR_i(nrow,i,model,label)
            if nuR:
                return nuR

    i=2
    nuR=get_nuR_i(nrow,i,model,label)
    if nuR:
        return nuR    
    else: 
        return 0

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [4]:
import numpy as np
from functools import reduce
import itertools
import pandas as pd
import os
from anomalies import anomaly

In [5]:
z=anomaly.free

In [6]:
def get_repeated(q,q_max=np.inf):
    #Normalize to positive minimum
    if q[0]<0 or (q[0]==0 and q[1]<0):
        q=-q
    #Divide by GCD
    GCD=np.gcd.reduce(q)
    q=(q/GCD).astype(int)
    if ( #not 0 in z and 
             0 not in [ sum(p) for p in itertools.permutations(q, 2) ] and #avoid vector-like
             q.size > np.unique(q).size and # check for at least a duplicated entry
             np.abs(q).max()<=q_max
           ):
        return q

In [137]:
import warnings
warnings.filterwarnings("ignore")

In [305]:
#TODO: inherit from free class
import sys
def _get_chiral(q,q_max=np.inf):
    #Normalize to positive minimum
    if q[0]<0 or (q[0]==0 and q[1]<0):
        q=-q
    #Divide by GCD
    GCD=np.gcd.reduce(q)
    q=(q/GCD).astype(int)
    if ( #not 0 in z and 
          0 not in [ sum(p) for p in itertools.permutations(q, 2) ] and #avoid vector-like and multiple 0's
          #q.size > np.unique(q).size and # check for at least a duplicated entry
          np.abs(q).max()<=q_max
           ):
        return q
class solutions(object):
    def __init__(self,nmin=-2,nmax=2,zmax=np.inf):
        self.nmin=nmin
        self.nmax=nmax
        self.zmax=zmax
        self.CALL=False

    def __call__(self,N,*args,**kwargs):
        self.CALL=True
        if N%2!=0: #odd
            N_l=(N-3)//2
            N_k=(N-1)//2
        else: #even
            N_l=N//2-1
            N_k=N_l
        r=range(self.nmin,self.nmax+1)
        self.ls=list(itertools.product( *(r for i in range(N_l)) ))
        self.ks=list(itertools.product( *(r for i in range(N_k)) ))
        return self.chiral(*args,**kwargs)
        
        
    def chiral(self,*args,**kwargs):
        if not self.CALL:
            sys.exit('Call the initialized object first:\n>>> s=solutions()\n>>> self(5)')
        self.list=[]
        solt=[]
        for l in self.ls:
            for k in self.ks:
                l=list(l)
                k=list(k)
                q=_get_chiral( z(l,k) )
                #print(z(l,k))
                if q is not None and list(q) not in self.list and list(-q) not in self.list:
                    self.list.append(list(q))
                    solt.append({'l':l,'k':k,'z':list(q)})
        return solt

In [306]:
s=solutions()

In [307]:
s.chiral()

SystemExit: Call the initialized object first:
>>> s=solutions()
>>> self(5)

In [308]:
s(5)

[{'l': [-2], 'k': [-1, 2], 'z': [2, 4, -7, -9, 10]},
 {'l': [-2], 'k': [2, -1], 'z': [1, 5, -7, -8, 9]}]

In [309]:
class dark(solutions):
    def chiral(self,X=False):
        m=0
        if X:
            m=1
        self.list=[]
        solt=[]
        for l in self.ls:
            for k in self.ks:
                l=list(l)
                k=list(k)
                q=_get_chiral( z(l,k) )
                #print(z(l,k))
                if (q is not None and 
                    list(q) not in self.list and list(-q) not in self.list and
                    q.size-np.unique(q).size>m
                   ):
                    self.list.append(list(q))
                    solt.append({'l':l,'k':k,'z':list(q)})
        return solt        

In [310]:
s=dark()

In [311]:
s(6,X=True)

[{'l': [-2, 2], 'k': [-2, -1], 'z': [1, 1, 1, -4, -4, 5]}]

In [ ]:
%%time
nmin=-32;nmax=32
sol=[]
solrep=[]
for l,k in [([i],[j,k]) for i in range(nmin,nmax)
    for j in range(nmin,nmax)
    for k in range(nmin,nmax)]:
    q=get_repeated( z(l,k) )
    if q is not None and list(q) not in sol and list(-q) not in sol:
        sol.append(list(q))
        if 1 in [ np.unique(p).size for p in itertools.permutations(q, 2) ]:
            solrep.append(q)
        print(l,k, q)

In [5]:
%%time
REBUILD=False
if not REBUILD:
    sys.exit('Already build')
ds=pd.DataFrame()
if os.path.exists('./solutions.json'):
    ds=pd.read_json('solutions.json')

sols={}
isol=[5,6,7,8,9]
for i in isol:
    if ds.empty:
        sols[i]=[]
    else:
        sols[i]=[l for l in ds[ds.n==i].solution.values]
q={}
l={}
k={}
for search in range(48):#000000):
    if search%500000==0: print("**** {} ******".format(search))
    qmax=32
    nmin=-10;nmin2=-10
    nmax=1+21;nmax2=1+15

    l1=np.random.randint(nmin,nmax);l2=np.random.randint(nmin,nmax)
    l3=np.random.randint(nmin,nmax);l4=np.random.randint(nmin,nmax)

    k1=np.random.randint(nmin2,nmax2);k2=np.random.randint(nmin2,nmax2)
    k3=np.random.randint(nmin2,nmax2);k4=np.random.randint(nmin2,nmax2)
    #Benchmark 
    #l1=1;k1=2;k2=-4
    l[5]=[l1]   ;k[5]=[k1,k2]
    l[6]=[l1,l2];k[6]=[k1,k2]
    l[7]=[l1,l2];k[7]=[k1,k2,k3]    
    l[8]=[l1,l2,l3];k[8]=[k1,k2,k3]    
    l[9]=[l1,l2,l3];k[9]=[k1,k2,k3,k4]    

    for i in isol:
        q[i]=z(l[i],k[i])[::-1] #q is a numpy array
        #Normalize to positive minimum
        if q[i][0]<0 or (q[i][0]==0 and q[i][1]<0):
            q[i]=-q[i]
        #Divide by GCD
        GCD=np.gcd.reduce(q[i])
        q[i]=(q[i]/GCD).astype(int)
        if ( #not 0 in q[i] and 
             not 0 in [ sum(p) for p in itertools.permutations(q[i], 2) ] and #avoid vector-like
             np.abs(q[i]).max()<=32
           ):
            s=q[i]
            if list(s) not in sols[i] and list(-s) not in sols[i]: #TODO → Already normalized to avoid -s
                print(i,l[i],k[i],s)
                sols[i].append(list(s))
                ds=ds.append({'n':i,'l':l[i],'k':k[i],'solution':list(s),'gcd':GCD},ignore_index=True)
                ds.to_json('tmp.json')
            #break

SystemExit: Already build

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
ds=pd.read_json('solutions.json')
ds['gcd']=ds.apply( lambda row:  np.gcd.reduce( z(row['l'],row['k']) ) ,axis='columns' )

In [7]:
ds[ds['n']==5]

,k,l,n,solution,gcd
0,"[-1, -2]",[2],5,"[2, 4, -7, -9, 10]",1
1,"[6, -2]",[16],5,"[7, 13, -25, -27, 32]",192
2,"[10, 12]",[-5],5,"[1, 14, -17, -18, 20]",300
3,"[-9, -3]",[3],5,"[5, 11, -18, -26, 28]",243
4,"[1, -4]",[-2],5,"[4, 9, -14, -25, 26]",1
5,"[3, 5]",[-3],5,"[7, 8, -18, -22, 25]",9
6,"[2, 6]",[-10],5,"[5, 8, -14, -26, 27]",48
7,"[1, 4]",[-1],5,"[1, 5, -7, -8, 9]",2
8,"[-5, 1]",[-3],5,"[7, 9, -20, -22, 26]",15
9,"[-5, 5]",[-4],5,"[7, 8, -17, -25, 27]",50


In [24]:
q=None
if q: print(1)

In [40]:
q=[11, 19, -32, -63, 65]

In [46]:
np.unique((2,2)).size

1

In [59]:
q[1]=11

In [60]:
q

[11, 11, -32, -63, 65]

[1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [82]:
%%time
1+1

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


2

In [91]:
%%time
nmin=-32;nmax=32
sol=[]
solrep=[]
for l,k in [([i],[j,k]) for i in range(nmin,nmax)
    for j in range(nmin,nmax)
    for k in range(nmin,nmax)]:
    q=get_repeated( z(l,k) )
    if q is not None and list(q) not in sol and list(-q) not in sol:
        sol.append(list(q))
        if 1 in [ np.unique(p).size for p in itertools.permutations(q, 2) ]:
            solrep.append(q)
        print(l,k, q)

/usr/local/lib/python3.7/dist-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in true_divide
  self.simplified=(zz/self.gcd).astype(int)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


CPU times: user 15.5 s, sys: 9.04 ms, total: 15.5 s
Wall time: 15.5 s


In [95]:
[0,1],[0,1]

[(0, 0), (1, 1)]

In [90]:
%%time
nmin=-32;nmax=32
sol=[]
solrep=[]
for l,k in [([i],[j,k]) for i in range(nmin,nmax)
    for j in range(nmin,nmax)
    for k in range(nmin,nmax)]:
    q=get_repeated( z(l,k) )
    if q is not None and list(q) not in sol and list(-q) not in sol:
        sol.append(list(q))
        if 1 in [ np.unique(p).size for p in itertools.permutations(q, 2) ]:
            solrep.append(q)
        print(l,k, q)

/usr/local/lib/python3.7/dist-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in true_divide
  self.simplified=(zz/self.gcd).astype(int)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


CPU times: user 15.4 s, sys: 8.8 ms, total: 15.4 s
Wall time: 15.4 s


In [65]:
solrep

[]

In [37]:
sol

[[11, 19, -32, -63, 65],
 [1, 14, -17, -18, 20],
 [7, 13, -25, -27, 32],
 [9, 43, -64, -65, 77],
 [9, 13, -23, -55, 56],
 [7, 8, -18, -22, 25],
 [2, 19, -22, -35, 36],
 [4, 13, -18, -34, 35],
 [8, 27, -41, -49, 55],
 [7, 26, -36, -55, 58],
 [7, 8, -17, -25, 27],
 [9, 11, -23, -32, 35],
 [3, 19, -24, -33, 35],
 [9, 26, -42, -48, 55],
 [2, 18, -23, -25, 28],
 [14, 38, -65, -68, 81],
 [7, 9, -20, -22, 26],
 [8, 44, -63, -65, 76],
 [1, 5, -7, -8, 9],
 [5, 6, -12, -21, 22],
 [5, 8, -14, -26, 27],
 [9, 25, -39, -51, 56],
 [5, 11, -18, -26, 28],
 [7, 19, -27, -64, 65],
 [19, 23, -49, -65, 72],
 [25, 29, -64, -81, 91],
 [5, 27, -34, -56, 58],
 [2, 4, -7, -9, 10],
 [13, 56, -76, -108, 115],
 [19, 58, -88, -115, 126],
 [17, 22, -50, -52, 63],
 [10, 11, -27, -28, 34],
 [3, 32, -39, -45, 49],
 [1, 63, -66, -78, 80],
 [1, 27, -31, -32, 35],
 [13, 123, -153, -175, 192],
 [17, 46, -80, -81, 98],
 [17, 21, -45, -57, 64],
 [4, 38, -43, -90, 91],
 [44, 61, -116, -189, 200],
 [20, 37, -72, -76, 91],
 [6,

In [33]:
q is not None# and list(q) not in sol and list(-q) not in sol

True

In [8]:
print(ds.shape)
#Small n first
ds=ds.sort_values('n').reset_index(drop=True)
#Remove zero charges
ds['solution']=ds['solution'].apply( lambda s: [s for s in s if s!=0] )
ds['sltn']=ds['solution'].astype(str)
ds=ds.drop_duplicates('sltn').drop(['sltn'],axis='columns').reset_index(drop=True)
ds.shape

(24326, 5)


(24326, 5)

In [9]:
ds[:1]

,k,l,n,solution,gcd
0,"[-1, -2]",[2],5,"[2, 4, -7, -9, 10]",1


In [10]:
SAVE=False
if SAVE:
    ds.to_json('solutions.json')

## Analysis
#### → is obtained with `<ALT GR>+i`

In [11]:
ds=label_solutions(ds,Dim=5)
ds6=label_solutions(ds,Dim=6)

In [12]:
ds.shape,ds6.shape

((24326, 10), (24326, 10))

## Conclusion

In [13]:
#extra=[[2, -3, 6, -12, -12, 24, 24, -29],
#       [1, 1, 1, 1, -4, 9, -10, -10, 11],
#       [3, 6, -12, -12, -12, 13, 14]]
#for s in extra:
#    cl=cl.append( tmp[ tmp['solution'].astype(str)==str(s) ],sort=True).reset_index(drop=True)
#del tmp
cl=filter_solution(ds,nmax=32)
cl[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn
0,"[-5, 0]","[1, -1]",6,"[2, -3, -10, 13, 13, -15]",2,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",2,0,0,0,{2: [13]}
1,"[-1, 0]","[2, -2]",6,"[1, -2, -3, 5, 5, -6]",4,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]",2,0,0,0,{2: [5]}
2,"[-4, 0]","[-1, 1]",6,"[3, -5, -12, 17, 17, -20]",1,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",2,0,0,0,{2: [17]}


In [14]:
cl6=filter_solution(ds6,nmax=32)
cl6[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn
0,"[-2, 1]","[-2, -1]",6,"[16, -19, -20, 26, 26, -29]",1,"[{'D→nu_R+2fi+2fj': [-29, 16, 26]}]",1,0,0,0,{2: [26]}
1,"[10, 5]","[10, -2]",6,"[17, -19, -23, 28, 28, -31]",200,"[{'D→nu_R+2fi+2fj': [-31, 17, 28]}]",1,0,0,0,{2: [28]}
2,"[2, 1]","[2, 1]",6,"[1, -4, -8, 14, 14, -17]",1,"[{'D→nu_R+2fi+2fj': [-8, 1, 14]}]",1,0,0,0,{2: [14]}


See https://stackoverflow.com/a/15070110/2268280

Tests

In [15]:
row=cl[cl['solution'].astype(str)=='[1, -2, -3, 5, 5, -6]'].iloc[-1]
assert get_nuR(row,'DarkDirac',Dim=5)==[5]
row=cl[cl['solution'].astype(str)=='[1, 1, -3, -4, 6, 6, -7]'].iloc[-1]
assert get_nuR(row,'DarkMajor',Dim=5)==[6]
row=cl6[cl6['solution'].astype(str)=='[1, 1, 1, -4, -4, 5]'].iloc[-1]
assert get_nuR(row,'DarkMajor',Dim=6)==[-4]
row=cl6[cl6['solution'].astype(str)=='[1, -4, -8, 14, 14, -17]'].iloc[-1]
assert get_nuR(row,'DarkDirac',Dim=6)==[14]
row=cl[cl['solution'].astype(str)=='[5, 5, 7, -8, -12, -12, 15]'].iloc[-1]
assert get_nuR(row,'DarkDirac',Dim=5)==[-12,5]
#X
row=cl[cl['solution'].astype(str)=='[1, 3, 3, 3, -5, -7, -7, 9]'].iloc[-1]
assert get_nuR(row,'XDirac',Dim=5)==[-7]
row=cl[cl['solution'].astype(str)=='[1, 1, -4, -5, 9, 9, 9, -10, -10]'].iloc[-1]
assert get_nuR(row,'XMajor',Dim=5)==[1]
row=cl6[cl6['solution'].astype(str)=='[3, 3, 3, -5, -5, -7, 8]'].iloc[-1]
assert get_nuR(row,'XDirac',Dim=6)==[-5]
row=cl6[cl6['solution'].astype(str)=='[1, 3, 3, 3, -5, -7, -7, 9]'].iloc[-1]
assert get_nuR(row,'XMajor',Dim=6)==[-7]
#Special cases
row=cl[cl['solution'].astype(str)=='[1, 5, -6, -6, 8, 13, -18, -18, 21]'].iloc[-1]
assert get_nuR(row,'DarkDirac',Dim=5)==[-18, -6]
row=cl[cl['solution'].astype(str)=='[4, -13, -13, -17, 19, 26, 26, -32]'].iloc[-1]
assert get_nuR(row,'DarkMajor',Dim=5)==[26]

In [16]:
cl['DD']=cl.apply(lambda row: get_nuR(row,'DarkDirac'),axis='columns')
cl['DM']=cl.apply(lambda row: get_nuR(row,'DarkMajor'),axis='columns')
cl['XD']=cl.apply(lambda row: get_nuR(row,'XDirac'),axis='columns')
cl['XM']=cl.apply(lambda row: get_nuR(row,'XMajor'),axis='columns')
cl6['DD']=cl6.apply(lambda row: get_nuR(row,'DarkDirac',Dim=6),axis='columns')
cl6['DM']=cl6.apply(lambda row: get_nuR(row,'DarkMajor',Dim=6),axis='columns')
cl6['XD']=cl6.apply(lambda row: get_nuR(row,'XDirac',Dim=6),axis='columns')
cl6['XM']=cl6.apply(lambda row: get_nuR(row,'XMajor',Dim=6),axis='columns')

In [17]:
cl[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
0,"[-5, 0]","[1, -1]",6,"[2, -3, -10, 13, 13, -15]",2,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",2,0,0,0,{2: [13]},[13],0,0,0
1,"[-1, 0]","[2, -2]",6,"[1, -2, -3, 5, 5, -6]",4,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]",2,0,0,0,{2: [5]},[5],0,0,0
2,"[-4, 0]","[-1, 1]",6,"[3, -5, -12, 17, 17, -20]",1,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",2,0,0,0,{2: [17]},[17],0,0,0


In [18]:
cl6[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
0,"[-2, 1]","[-2, -1]",6,"[16, -19, -20, 26, 26, -29]",1,"[{'D→nu_R+2fi+2fj': [-29, 16, 26]}]",1,0,0,0,{2: [26]},[26],0,0,0
1,"[10, 5]","[10, -2]",6,"[17, -19, -23, 28, 28, -31]",200,"[{'D→nu_R+2fi+2fj': [-31, 17, 28]}]",1,0,0,0,{2: [28]},[28],0,0,0
2,"[2, 1]","[2, 1]",6,"[1, -4, -8, 14, 14, -17]",1,"[{'D→nu_R+2fi+2fj': [-8, 1, 14]}]",1,0,0,0,{2: [14]},[14],0,0,0


In [19]:
cl6[cl6['XDirac']>0]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
38,"[2, 3, -1]","[3, 1]",7,"[3, 3, 3, -5, -5, -7, 8]",1,"[{'X→nu_R+2fi+2fj': [-7, -5, 8]}]",0,0,1,0,"{2: [-5], 3: [3]}",0,0,[-5],0
509,"[-9, -1, 4]","[4, 8, 6]",8,"[7, 8, 8, 8, -18, -18, -23, 28]",80,"[{'X→nu_R+2fi+2fj': [-23, -18, 28]}]",0,0,1,0,"{2: [-18], 3: [8]}",0,0,[-18],0
510,"[10, 11, -2]","[10, 20, 15]",8,"[7, -8, -18, -18, 20, 20, 20, -23]",3450,"[{'X→nu_R+2fi+2fj': [-18, -8, 7]}]",0,0,1,0,"{2: [-18], 3: [20]}",0,0,[-18],0
511,"[3, 7, 3]","[17, 0, 17]",8,"[3, 3, 3, -7, 17, -23, -23, 27]",4624,"[{'X→nu_R+2fi+2fj': [-23, -7, 17]}]",0,0,1,0,"{2: [-23], 3: [3]}",0,0,[-23],0
799,"[4, -4, 5, 2]","[7, 0, 2]",9,"[4, 7, -8, 9, -16, -16, -16, 18, 18]",46,"[{'D→nu_R+2fi+2fj': [-16, 7, 18]}, {'D→nu_R+4fi': [-16, 4]}, {'X→nu_R+2fi+2fj': [-8, 7, 18]}]",1,1,1,0,"{2: [18], 3: [-16]}",[18],[-16],[18],0
899,"[2, -8, -6, -2]","[10, -5, -7]",9,"[1, 1, 1, 4, -9, -10, -10, 11, 11]",128,"[{'D→nu_R+2fi+2fj': [-10, 1, 4]}, {'X→nu_R+2fi+2fj': [-10, 4, 11]}]",1,0,1,0,"{2: [-10, 11], 3: [1]}",[-10],0,"[-10, 11]",0
900,"[10, 12, 3, 6]","[6, -4, 7]",9,"[6, 6, 6, -8, -10, 13, -16, -16, 19]",251,"[{'D→nu_R+2fi+2fj': [-16, 6, 13]}, {'X→nu_R+2fi+2fj': [-16, -8, 13]}]",1,0,1,0,"{2: [-16], 3: [6]}",[6],0,[-16],0
901,"[-1, 5, 2, 11]","[-2, 1, -3]",9,"[3, 3, 3, -4, -4, 8, -11, -11, 13]",16,"[{'D→nu_R+2fi+2fj': [-11, -4, 13]}, {'X→nu_R+2fi+2fj': [-11, -4, 8]}]",1,0,1,0,"{2: [-4, -11], 3: [3]}",[-4],0,"[-11, -4]",0
902,"[12, 14, 10, 6]","[8, 16, -4]",9,"[3, 4, -10, -10, -10, 12, 12, 13, -14]",1280,"[{'D→nu_R+2fi+2fj': [-10, 4, 12]}, {'X→nu_R+2fi+2fj': [-14, 12, 13]}]",1,0,1,0,"{2: [12], 3: [-10]}",[12],0,[12],0
2324,"[-10, 6, 1, -4]","[18, 13, 6]",9,"[2, -7, -8, -8, -8, 18, 18, 18, -25]",608,"[{'D→nu_R+4fi': [-8, 2]}, {'X→nu_R+2fi+2fj': [-7, 2, 18]}, {'X→nu_R+2fi+2fj': [-8, -7, 2]}]",0,1,2,0,"{3: [-8, 18]}",0,[-8],"[-8, 18]",0


There are no solution for $X$-Majorana for $n<9$

In [20]:
cl.to_json('cl.json')

In [21]:
cl6.to_json('cl6.json')

In [120]:
cl[cl['XMajor']>0]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
1070,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1,"{2: [1, -10], 3: [9]}",[9],0,0,[1]
1596,"[-2, 6, 0, 1]","[-2, 1, 2]",9,"[2, 2, -6, -7, -9, 10, 10, 10, -12]",10,"[{'D→nu_R+fi+fj': [-12, 2, 10]}, {'X→nu_R+2fi': [2, -6]}]",1,0,0,1,"{2: [2], 3: [10]}",[10],0,0,[2]
1597,"[-10, -8, 4, 9]","[8, 6, -2]",9,"[1, 1, 1, 2, -6, -9, 11, 11, -12]",20,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'X→nu_R+2fi': [11, -6]}]",1,0,0,1,"{2: [11], 3: [1]}",[1],0,0,[11]
1598,"[-9, -4, 0, 4]","[2, -6, 9]",9,"[2, -6, -9, -9, -12, 21, 21, 21, -29]",264,"[{'D→nu_R+fi+fj': [-12, -9, 21]}, {'X→nu_R+2fi': [-9, -6]}]",1,0,0,1,"{2: [-9], 3: [21]}",[21],0,0,[-9]
4488,"[-2, -10, -1, 8]","[-4, 13, 16]",9,"[4, 4, 4, -7, -8, -9, 14, 14, -16]",478,"[{'D→nu_R+2fi': [14, -7]}, {'X→nu_R+2fi': [14, -9]}]",0,1,0,1,"{2: [14], 3: [4]}",0,[14],0,[14]
4946,"[-6, 7, 1, 11]","[17, 10, 4]",9,"[3, 3, 3, 7, 11, -16, -17, -17, 23]",170,"[{'X→nu_R+2fi': [-17, 7]}]",0,0,0,1,"{2: [-17], 3: [3]}",0,0,0,[-17]
4947,"[-6, -10, -3, 12]","[18, 2, 9]",9,"[3, -8, 12, 12, 12, -18, -18, -19, 24]",853,"[{'X→nu_R+2fi': [-18, 3]}]",0,0,0,1,"{2: [-18], 3: [12]}",0,0,0,[-18]
4948,"[3, -7, -5, 12]","[-7, 13, 17]",9,"[1, 1, 3, -7, -7, -7, 19, 24, -27]",406,"[{'X→nu_R+2fi': [1, 3]}]",0,0,0,1,"{2: [1], 3: [-7]}",0,0,0,[1]
4949,"[-1, -3, 7, 13]","[8, 9, 4]",9,"[5, 5, -11, 13, -15, -15, -15, 16, 17]",60,"[{'X→nu_R+2fi': [5, 5]}]",0,0,0,1,"{2: [5], 3: [-15]}",0,0,0,[5]
4950,"[2, -8, 7, 9]","[9, -9, -8]",9,"[1, -7, 9, 9, 9, -11, -11, -16, 17]",320,"[{'X→nu_R+2fi': [-11, 1]}]",0,0,0,1,"{2: [-11], 3: [9]}",0,0,0,[-11]


In [41]:
nmax=10
clmax=cl[cl['solution'].apply(lambda l: np.abs(l).max() if isinstance(l,list) else [])<=nmax].reset_index(drop=True)

In [42]:
clmax

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
0,"[-1, 0]","[2, -2]",6,"[1, -2, -3, 5, 5, -6]",4,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]",2,0,0,0,{2: [5]},[5],0,0,0
1,"[1, 0, -1]","[-4, -6]",7,"[1, 1, -3, -4, 6, 6, -7]",3,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1,1,0,0,"{2: [1, 6]}","[1, 6]",[6],0,0
2,"[-1, -4, -3]","[2, 0]",7,"[1, 3, -4, 5, -6, -6, 7]",3,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,1,0,0,{2: [-6]},[-6],[-6],0,0
3,"[-3, -4, -3]","[-1, 0]",7,"[1, 2, 2, -3, -3, -3, 4]",6,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,0,0,0,"{2: [2], 3: [-3]}",[-3],0,0,0
4,"[-2, -11, -8]","[1, -1]",7,"[1, 1, -3, -6, 8, 9, -10]",1,"[{'D→nu_R+fi+fj': [-10, 1, 9]}]",1,0,0,0,{2: [1]},[1],0,0,0
5,"[2, -7, -6]","[-1, 1]",7,"[2, 3, 3, -4, -5, -6, 7]",3,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]",1,0,0,0,{2: [3]},[3],0,0,0
6,"[3, 4, -4]","[4, -5]",7,"[1, 1, -4, -4, 7, 8, -9]",1,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]",1,0,0,0,"{2: [1, -4]}",[1],0,0,0
7,"[3, -1, 0]","[1, -2, 1]",8,"[1, 2, 3, 5, -6, -6, -9, 10]",1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,1,0,0,{2: [-6]},[-6],[-6],0,0
8,"[1, 0, -2]","[1, -6, -9]",8,"[1, 1, 2, 3, -4, -4, -5, 6]",2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]",1,1,0,0,"{2: [1, -4]}","[-4, 1]",[-4],0,0
9,"[10, -5, 5]","[5, 0, -5]",8,"[2, -5, -5, -5, 7, 8, 8, -10]",625,"[{'D→nu_R+fi+fj': [-10, 2, 8]}, {'X→nu_R+fi+fj': [-10, 7, 8]}]",1,0,1,0,"{2: [8], 3: [-5]}",[8],0,[8],0


In [43]:
kk=clmax[['l','k','gcd','n','solution','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ])   )

In [123]:
cat solutions.tex | head -n5

\begin{tabular}{rlllrllll}
\toprule
n &             l &                 k &                                 solution & n &            DD &        DM &        XD &    XM \\
\midrule
6 &       (2, -2) &           (-1, 0) &                    (1, -2, -3, 5, 5, -6) & 6 &           (5) &         0 &         0 &     0 \\


In [45]:
nmax=10
clmax6=cl6[cl6['solution'].apply(lambda l: np.abs(l).max() if isinstance(l,list) else [])<=nmax].reset_index(drop=True)

In [46]:
clmax6

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
0,"[4, -2]","[-1, 1]",6,"[1, 1, 1, -4, -4, 5]",3,"[{'D→nu_R+4fi': [-4, 1]}]",0,1,0,0,"{2: [-4], 3: [1]}",0,0,0,0
1,"[-1, 1, 3]","[7, 5]",7,"[2, 2, -4, 7, -8, -8, 9]",1,"[{'D→nu_R+2fi+2fj': [-8, 2, 7]}, {'D→nu_R+4fi': [-8, 2]}]",1,1,0,0,"{2: [2, -8]}",0,0,0,0
2,"[1, 0, -1]","[-4, -6]",7,"[1, 1, -3, -4, 6, 6, -7]",3,"[{'D→nu_R+2fi+2fj': [-4, 1, 6]}]",1,0,0,0,"{2: [1, 6]}",0,0,0,0
3,"[-1, -4, -3]","[2, 0]",7,"[1, 3, -4, 5, -6, -6, 7]",3,"[{'D→nu_R+2fi+2fj': [-6, -4, 7]}]",1,0,0,0,{2: [-6]},0,0,0,0
4,"[3, 4, -4]","[4, -5]",7,"[1, 1, -4, -4, 7, 8, -9]",1,"[{'D→nu_R+4fi': [-4, 1]}]",0,1,0,0,"{2: [1, -4]}",0,0,0,0
5,"[2, 3, -1]","[3, 1]",7,"[3, 3, 3, -5, -5, -7, 8]",1,"[{'X→nu_R+2fi+2fj': [-7, -5, 8]}]",0,0,1,0,"{2: [-5], 3: [3]}",0,0,0,0
6,"[-2, -4, -2]","[1, 0, 1]",8,"[1, -2, -2, 4, 5, -7, -7, 8]",1,"[{'D→nu_R+2fi+2fj': [-7, -2, 8]}]",1,0,0,0,"{2: [-2, -7]}",0,0,0,0
7,"[2, 1, 0]","[-1, 3, -4]",8,"[1, -3, -4, -5, 6, 6, 8, -9]",2,"[{'D→nu_R+2fi+2fj': [-4, 1, 6]}]",1,0,0,0,{2: [6]},0,0,0,0
8,"[-1, 2, -1]","[3, -2, -3]",8,"[1, -3, -5, -6, 7, 8, 8, -10]",3,"[{'D→nu_R+2fi+2fj': [-5, 1, 8]}]",1,0,0,0,{2: [8]},0,0,0,0
9,"[-5, 5, 4]","[1, 3, -1]",8,"[1, 2, 2, 4, -5, -5, -7, 8]",4,"[{'D→nu_R+2fi+2fj': [-5, 2, 4]}]",1,0,0,0,"{2: [2, -5]}",0,0,0,0


## Appendix

### Analysis of the best solutions

In [23]:
dsf={}
for i in [2,3,4]:
    dsf['{}'.format(i)]=extract_multiplets(ds,i,greater_than=0)
    dsf['{}→x→x:0'.format(i)]=dsf['{}'.format(i)][dsf['{}'.format(i)]['DarkDirac']>0
                        ].sort_values('DarkDirac',ascending=False).reset_index(drop=True)    
    dsf['{}→2*x:0'.format(i)]=dsf['{}'.format(i)][dsf['{}'.format(i)]['DarkMajor']>0
                        ].sort_values('DarkMajor',ascending=False).reset_index(drop=True)
    if i>2:
        dsf['x→x→x:{}'.format(i)]=dsf['{}'.format(i)][dsf['{}'.format(i)]['XDirac']>0
                        ].sort_values('XDirac',ascending=False).reset_index(drop=True)        
        dsf['nu_R→2*x:{}'.format(i)]=dsf['{}'.format(i)][dsf['{}'.format(i)]['XMajor']>0
                        ].sort_values('XMajor',ascending=False).reset_index(drop=True)        
#No higher solutions found with 9 fields

dsf['2→2']=extract_multiplets(ds,2,greater_than=1)
dsf['3→3']=extract_multiplets(ds,3,greater_than=1)
dsf['2→3']=extract_multiplets(dsf['2'],3,greater_than=0)
dsf['2→4']=extract_multiplets(dsf['2'],4,greater_than=0)
dsf['2→2→2']=extract_multiplets(ds,2,greater_than=2)
dsf['2→2→3']=extract_multiplets(dsf['2→2'],3,greater_than=0)
#No other combinations found 

dsf['2→2']['2→2sum']=dsf['2→2']['solution'].apply(
                  lambda f: repeated(np.asarray(f))).apply(
                  lambda d: d[2]).apply(sum)
dsf['3→3']['3→3sum']=dsf['3→3']['solution'].apply(
                  lambda f: repeated(np.asarray(f))).apply(
                  lambda d: d[3]).apply(sum)
dsf['2→3']['2→3sum']=dsf['2→3']['solution'].apply(
                  lambda f: repeated(np.asarray(f))).apply(
                  lambda d: d[2]+d[3]).apply(sum)
dsf['2→4']['2→4sum']=dsf['2→4']['solution'].apply(
                  lambda f: repeated(np.asarray(f))).apply(
                  lambda d: d[2]+d[4]).apply(sum)

Dark Dirac

In [24]:
dsf['2→2→x:0']=dsf['2→2'][dsf['2→2'].apply(lambda row: -row['2→2sum'] in row['solution'],axis='columns')]
dsf['3→3→x:0']=dsf['3→3'][dsf['3→3'].apply(lambda row: -row['3→3sum'] in row['solution'],axis='columns')]
dsf['2→3→x:0']=dsf['2→3'][dsf['2→3'].apply(lambda row: -row['2→3sum'] in row['solution'],axis='columns')]
dsf['2→4→x:0']=dsf['2→4'][dsf['2→4'].apply(lambda row: -row['2→4sum'] in row['solution'],axis='columns')]

dsf['2→2→2:0']=dsf['2→2→2'][dsf['2→2→2']['solution'].apply(lambda f: repeated(np.asarray(f))).str[2].apply(sum)==0]
dsf['2→2→3:0']=dsf['2→2→3'][dsf['2→2→3']['solution'].apply(
                  lambda f: repeated(np.asarray(f))).apply(
                  lambda d: d[2]+d[3]).apply(sum)==0]

Dark Majorana

In [25]:
dsf['2→2*2:0']=dsf['2→2'][ dsf['2→2']['nu_R'].apply(lambda l: [d.get('D→nu_R+2fi') for d in l if d.get('D→nu_R+2fi')]).apply(len)>1 ]
dsf['3→2*2:0']=dsf['2→3'][ dsf['2→3']['nu_R'].apply(lambda l: [d.get('D→nu_R+2fi') for d in l if d.get('D→nu_R+2fi')]).apply(len)>1 ]
dsf['3→2*3:0']=dsf['3→3'][ dsf['3→3']['nu_R'].apply(lambda l: [d.get('D→nu_R+2fi') for d in l if d.get('D→nu_R+2fi')]).apply(len)>1 ]
dsf['4→2*3:0']=dsf['2→4'][ dsf['2→4']['nu_R'].apply(lambda l: [d.get('D→nu_R+2fi') for d in l if d.get('D→nu_R+2fi')]).apply(len)>1 ]

X Dirac

In [26]:
dsf['2→x→x:3']=dsf['2→3'][ dsf['2→3'].apply(  lambda row: [ True for y in [sum(x.get('X→nu_R+fi+fj')) 
                                                  for x in row['nu_R'] if x.get('X→nu_R+fi+fj')] 
                                                  if -y in row['solution'] ], #-y in Triplet already checked
                                  axis='columns'   ).apply(any) ]
dsf['3→x→x:3']=dsf['3→3'][ dsf['3→3'].apply(  lambda row: [ True for y in [sum(x.get('X→nu_R+fi+fj')) 
                                                  for x in row['nu_R'] if x.get('X→nu_R+fi+fj')] 
                                                  if -y in row['solution'] ], #-y in Triplet already checked
                                  axis='columns'   ).apply(any) ]

In [27]:
dsf['2→2→x:3']=dsf['2→x→x:3'][dsf['2→x→x:3']['solution'].apply(lambda f: repeated(np.asarray(f))).str[2].apply(len)>1]

In [28]:
dsf['nu_R→2*2:3']=dsf['nu_R→2*x:3'][dsf['nu_R→2*x:3']['solution'].apply(lambda f: repeated(np.asarray(f))).str[2].apply(len)>1]

X Majorana

In [29]:
dsf['nu_R→2*x:3'].shape

(14, 10)

In [30]:
mergeds=ds.copy()
mergeds['sltn']=ds['solution'].astype(str)

Check solutions:

Dark Dirac & Majorana

In [31]:
pd.DataFrame([
{'sltn':'[1, -2, -3, 5, 5, -6]'},    
{'sltn':'[1, 2, 2, -3, -3, -3, 4]'},
{'sltn':'[1, 1, -3, -4, 6, 6, -7]'},
{'sltn':'[1, 3, -4, 5, -6, -6, 7]'},
{'sltn':'[2, -3, 6, -12, -12, 24, 24, -29]'},    
{'sltn':'[1, 1, 1, 1, -4, 9, -10, -10, 11]'},
{'sltn':'[1, 1, 1, 2, 5, -6, -6, -6, 8]'},
{'sltn':'[1, 1, -4, -5, 9, 9, 9, -10, -10]'},
{'sltn':'[3, 3, -4, 5, 5, -6, -8, -8, 10]'},
{'sltn':'[3, 6, -12, -12, -12, 13, 14]'} 
]).merge(mergeds,on='sltn',how='left').drop('sltn',axis='columns')

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
0,"[-1, 0]","[2, -2]",6,"[1, -2, -3, 5, 5, -6]",4,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]",2,0,0,0
1,"[-3, -4, -3]","[-1, 0]",7,"[1, 2, 2, -3, -3, -3, 4]",6,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,0,0,0
2,"[1, 0, -1]","[-4, -6]",7,"[1, 1, -3, -4, 6, 6, -7]",3,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1,1,0,0
3,"[-1, -4, -3]","[2, 0]",7,"[1, 3, -4, 5, -6, -6, 7]",3,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,1,0,0
4,"[-2, 7, 3]","[-8, -3, 12]",8,"[2, -3, 6, -12, -12, 24, 24, -29]",420,"[{'D→nu_R+2fi': [-12, 6]}, {'D→nu_R+2fi': [24, -12]}]",0,2,0,0
5,"[10, 11, 2, -1]","[-5, 0, -5]",9,"[1, 1, 1, 1, -4, 9, -10, -10, 11]",275,"[{'D→nu_R+fi+fj': [-10, 1, 9]}]",1,0,0,0
6,"[-1, 1, 4, 6]","[-3, 0, 1]",9,"[1, 1, 1, 2, 5, -6, -6, -6, 8]",4,"[{'D→nu_R+fi+fj': [-6, 1, 5]}]",1,0,0,0
7,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1
8,"[10, -6, 7, 4]","[5, 2, -3]",9,"[3, 3, -4, 5, 5, -6, -8, -8, 10]",3,"[{'D→nu_R+fi+fj': [-8, 3, 5]}]",1,0,0,0
9,"[-2, -6, 12]","[4, -5]",7,"[3, 6, -12, -12, -12, 13, 14]",120,"[{'D→nu_R+2fi': [-12, 6]}]",0,1,0,0


X Dirac & Majorana

In [32]:
pd.DataFrame([
{'sltn':'[2, -5, -5, -5, 7, 8, 8, -10]'},
{'sltn':'[2, 2, 2, -3, -3, 4, -5, -5, 6]'},
{'sltn':'[1, 1, -4, -5, 9, 9, 9, -10, -10]'}    
]).merge(mergeds,on='sltn',how='left').drop('sltn',axis='columns')

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
0,"[10, -5, 5]","[5, 0, -5]",8,"[2, -5, -5, -5, 7, 8, 8, -10]",625,"[{'D→nu_R+fi+fj': [-10, 2, 8]}, {'X→nu_R+fi+fj': [-10, 7, 8]}]",1,0,1,0
1,"[1, -3, -4, -5]","[3, -1, 0]",9,"[2, 2, 2, -3, -3, 4, -5, -5, 6]",4,"[{'X→nu_R+fi+fj': [-5, -3, 6]}]",0,0,1,0
2,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1


Best solutions:

* [1, 2, 2, -3, -3, -3, 4]    → $S'=4+1$ avoid two massles states

In [33]:
dsf['2→3→x:0'].sort_values('n')[:1]#[( (dsf['2→2→x:0']['DarkDirac']>0) & (dsf['2→2→x:0']['DarkMajor']>0))].sort_values('n')[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,2→3sum
2,"[-3, -4, -3]","[-1, 0]",7,"[1, 2, 2, -3, -3, -3, 4]",6,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,0,0,0,-1


* [1, -2, -3, 5, 5, -6]  - Dirac-Dirac

In [34]:
(dsf['2→x→x:0'][(dsf['2→x→x:0']['DarkDirac']>1)]).sort_values('n')[:4]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
389,"[-4, 0]","[-1, 1]",6,"[3, -5, -12, 17, 17, -20]",1,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",2,0,0,0
241,"[-1, 0]","[2, -2]",6,"[1, -2, -3, 5, 5, -6]",4,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]",2,0,0,0
70,"[-5, 0]","[1, -1]",6,"[2, -3, -10, 13, 13, -15]",2,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",2,0,0,0
383,"[4, 7]","[21, 18]",6,"[3, -4, -21, 25, 25, -28]",7749,"[{'D→nu_R+fi+fj': [-28, 3, 25]}, {'D→nu_R+fi+fj': [-21, -4, 25]}]",2,0,0,0


* [1, 1, -3, -4, 6, 6, -7] - Dirac+Majorana

In [35]:
dsf['2→2→x:0'][( (dsf['2→2→x:0']['DarkDirac']>0) & (dsf['2→2→x:0']['DarkMajor']>0))].sort_values('n')[:3]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,2→2sum
10,"[1, 0, -1]","[-4, -6]",7,"[1, 1, -3, -4, 6, 6, -7]",3,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1,1,0,0,7
425,"[1, 0, -2]","[1, -6, -9]",8,"[1, 1, 2, 3, -4, -4, -5, 6]",2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]",1,1,0,0,-3
1592,"[1, 3, 1, -4]","[-6, 18, 14]",9,"[3, -6, -6, -7, -7, 11, 13, 28, -29]",41,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-7, -6, 13]}]",1,1,0,0,-13


In [36]:
dsf['2→4→x:0'].sort_values('n')

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,2→4sum
4,"[10, 11, 2, -1]","[-5, 0, -5]",9,"[1, 1, 1, 1, -4, 9, -10, -10, 11]",275,"[{'D→nu_R+fi+fj': [-10, 1, 9]}]",1,0,0,0,-9
5,"[-3, 6, -2, 4]","[-3, -7, -6]",9,"[3, 9, 9, 11, -12, -12, -12, -12, 16]",60,"[{'D→nu_R+fi+fj': [-12, 3, 9]}]",1,0,0,0,-3


* [1, 1, -4, -5, 9, 9, 9, -10, -10] Already studied

In [37]:
dsf['2→2→3:0'].sort_values('n')

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
6,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1
10,"[4, -2, 0, 11]","[10, 6, -4]",9,"[4, -5, -5, -6, -6, 11, 11, 11, -15]",392,"[{'D→nu_R+fi+fj': [-6, -5, 11]}, {'D→nu_R+fi+fj': [-15, 4, 11]}]",2,0,0,0
17,"[1, 12, 0, -3]","[1, 0, -1]",9,"[1, 1, -3, -8, 11, 11, 11, -12, -12]",14,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'D→nu_R+fi+fj': [-8, -3, 11]}]",2,0,0,0
20,"[2, -2, 0, 2]","[-6, 0, 6]",9,"[4, 4, -5, 15, 15, -19, -19, -19, 24]",16,"[{'D→nu_R+fi+fj': [-19, 4, 15]}, {'D→nu_R+fi+fj': [-19, -5, 24]}]",2,0,0,0


* [3, 3, -4, 5, 5, -6, -8, -8, 10] → -6 -4 + 10 Not a BUG because is an independent right handed neutrino with another S'

In [38]:
dsf['2→2→2:0'].sort_values('n')[:4]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
40,"[2, 4, -3, -7]","[2, 18, 16]",9,"[1, 1, -7, -8, 15, 20, 20, -21, -21]",384,"[{'D→nu_R+fi+fj': [-21, 1, 20]}]",1,0,0,0
45,"[9, -9, 0, -6]","[6, 0, -6]",9,"[2, -3, -3, -13, -13, 16, 16, 24, -26]",486,"[{'D→nu_R+fi+fj': [-13, -3, 16]}]",1,0,0,0
59,"[10, -6, 7, 4]","[5, 2, -3]",9,"[3, 3, -4, 5, 5, -6, -8, -8, 10]",3,"[{'D→nu_R+fi+fj': [-8, 3, 5]}]",1,0,0,0
89,"[3, -4, -6, 8]","[-5, -7, -2]",9,"[5, 5, -10, -14, 16, 16, -21, -21, 24]",50,"[{'D→nu_R+fi+fj': [-21, 5, 16]}]",1,0,0,0


* [1, 1, 1, 2, 5, -6, -6, -6, 8] Too many massles states

In [39]:
dsf['3→3→x:0'].sort_values('n')

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,3→3sum
4,"[-1, 1, 4, 6]","[-3, 0, 1]",9,"[1, 1, 1, 2, 5, -6, -6, -6, 8]",4,"[{'D→nu_R+fi+fj': [-6, 1, 5]}]",1,0,0,0,-5
5,"[5, 2, -2, 3]","[-8, 6, -3]",9,"[1, -3, -3, -3, -5, 8, 8, 8, -11]",301,"[{'D→nu_R+fi+fj': [-5, -3, 8]}]",1,0,0,0,5


* [1, 1, 1, 1, -4, 9, -10, -10, 11] Too many massless states NEW SOLUTION

In [40]:
dsf['4→x→x:0'].sort_values('n')

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
0,"[-5, 5, -10, -5]","[-3, 0, 4]",9,"[4, 4, 4, 4, 9, -13, -16, -19, 23]",200,"[{'D→nu_R+fi+fj': [-13, 4, 9]}]",1,0,0,0
1,"[2, 0, 3, -6]","[-3, -9, -6]",9,"[3, 3, 3, 3, 6, -9, -11, -16, 18]",24,"[{'D→nu_R+fi+fj': [-9, 3, 6]}]",1,0,0,0
2,"[10, 11, 2, -1]","[-5, 0, -5]",9,"[1, 1, 1, 1, -4, 9, -10, -10, 11]",275,"[{'D→nu_R+fi+fj': [-10, 1, 9]}]",1,0,0,0
3,"[-3, 6, -2, 4]","[-3, -7, -6]",9,"[3, 9, 9, 11, -12, -12, -12, -12, 16]",60,"[{'D→nu_R+fi+fj': [-12, 3, 9]}]",1,0,0,0
4,"[6, 11, 6, -3]","[-3, 7, 3]",9,"[3, 3, 3, 3, 9, -12, -13, -14, 18]",70,"[{'D→nu_R+fi+fj': [-12, 3, 9]}]",1,0,0,0
5,"[-2, -5, -2, 3]","[-4, -2, -4]",9,"[2, 2, 2, 2, 7, -8, -9, -11, 13]",8,"[{'D→nu_R+fi+fj': [-9, 2, 7]}]",1,0,0,0


In [41]:
#Already found
dsf['3→x→x:0'].sort_values('n')[:4]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
197,"[-1, 0, 5]","[-1, 1]",7,"[2, 22, 25, -27, -27, -27, 32]",1,"[{'D→nu_R+fi+fj': [-27, 2, 25]}]",1,0,0,0
195,"[1, -1, 1]","[3, 0]",7,"[2, 8, 9, -11, -11, -11, 14]",1,"[{'D→nu_R+fi+fj': [-11, 2, 9]}]",1,0,0,0
196,"[1, -1, 1]","[-6, 0]",7,"[1, 16, 18, -19, -19, -19, 22]",2,"[{'D→nu_R+fi+fj': [-19, 1, 18]}]",1,0,0,0
198,"[-3, -4, -3]","[-1, 0]",7,"[1, 2, 2, -3, -3, -3, 4]",6,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,0,0,0


Majorana specific

In [42]:
[c for c in dsf.keys() if c.find('*')>-1]

['2→2*x:0',
 '3→2*x:0',
 'nu_R→2*x:3',
 '4→2*x:0',
 'nu_R→2*x:4',
 '2→2*2:0',
 '3→2*2:0',
 '3→2*3:0',
 '4→2*3:0',
 'nu_R→2*2:3']

* [1, 3, -4, 5, -6, -6, 7] → Two massless states

In [43]:
dsf['2→2*x:0'][ dsf['2→2*x:0']['DarkDirac']==1 ].sort_values(['n'])[:6]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
536,"[6, 11, 9]","[6, 3]",7,"[4, -5, 7, 11, -22, -22, 27]",63,"[{'D→nu_R+2fi': [-22, 11]}, {'D→nu_R+fi+fj': [-22, -5, 27]}]",1,1,0,0
537,"[-1, -4, -3]","[2, 0]",7,"[1, 3, -4, 5, -6, -6, 7]",3,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,1,0,0
539,"[-4, -6, -4]","[9, 12]",7,"[8, 9, 10, -14, -18, -18, 23]",232,"[{'D→nu_R+2fi': [-18, 9]}, {'D→nu_R+fi+fj': [-18, 8, 10]}]",1,1,0,0
543,"[-5, -9, 7]","[5, 15]",7,"[2, -4, 7, 9, -18, -18, 22]",625,"[{'D→nu_R+2fi': [-18, 9]}, {'D→nu_R+fi+fj': [-18, -4, 22]}]",1,1,0,0
544,"[10, 0, 15]","[-5, -10]",7,"[9, 11, 13, -17, -22, -22, 28]",1250,"[{'D→nu_R+2fi': [-22, 11]}, {'D→nu_R+fi+fj': [-22, 9, 13]}]",1,1,0,0
679,"[1, 0, -1]","[-4, -6]",7,"[1, 1, -3, -4, 6, 6, -7]",3,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1,1,0,0


* [2, -3, 6, -12, -12, 24, 24, -29] → Four massless states

In [44]:
dsf['2→2*2:0'].sort_values(['n'])

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,2→2sum
326,"[-2, 7, 3]","[-8, -3, 12]",8,"[2, -3, 6, -12, -12, 24, 24, -29]",420,"[{'D→nu_R+2fi': [-12, 6]}, {'D→nu_R+2fi': [24, -12]}]",0,2,0,0,12
656,"[2, -1, -2, -3]","[-2, -1, -7]",9,"[1, -2, -2, -4, -7, 8, 8, 9, -11]",16,"[{'D→nu_R+2fi': [-2, 1]}, {'D→nu_R+fi+fj': [-7, -2, 9]}, {'D→nu_R+2fi': [8, -4]}]",1,2,0,0,6
723,"[-2, 2, -2, -8]","[14, 15, -9]",9,"[4, 7, 9, -10, -14, -14, 20, 20, -22]",64,"[{'D→nu_R+2fi': [-14, 7]}, {'D→nu_R+2fi': [20, -10]}]",0,2,0,0,6
966,"[4, -2, -4, -6]","[-4, -1, -5]",9,"[1, -4, 7, 8, 8, -9, -14, -14, 17]",16,"[{'D→nu_R+2fi': [8, -4]}, {'D→nu_R+fi+fj': [-9, 1, 8]}, {'D→nu_R+2fi': [-14, 7]}]",1,2,0,0,-6
979,"[-1, -4, -10, 6]","[-7, 1, -4]",9,"[2, -3, 4, 6, 6, -8, -8, -13, 14]",132,"[{'D→nu_R+2fi': [6, -3]}, {'D→nu_R+fi+fj': [-8, 2, 6]}, {'D→nu_R+2fi': [-8, 4]}]",1,2,0,0,-2
1098,"[-8, 8, 4, -6]","[2, -5, 14]",9,"[1, -2, -2, -7, 8, -9, 14, 14, -17]",704,"[{'D→nu_R+2fi': [-2, 1]}, {'D→nu_R+2fi': [14, -7]}]",0,2,0,0,12
1180,"[-6, 4, -6, 4]","[-5, -6, -9]",9,"[3, 5, -6, -6, 7, 8, -14, -14, 17]",120,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+2fi': [-14, 7]}]",0,2,0,0,-20
1871,"[3, 11, -9, -8]","[8, 15, -3]",9,"[4, 5, 6, -8, -8, 11, -12, -12, 14]",1821,"[{'D→nu_R+2fi': [-8, 4]}, {'D→nu_R+2fi': [-12, 6]}]",0,2,0,0,-20
1919,"[-8, 14, 4, 7]","[-10, 1, 4]",9,"[4, 6, 7, -10, -12, -12, 20, 20, -23]",746,"[{'D→nu_R+2fi': [-12, 6]}, {'D→nu_R+2fi': [20, -10]}]",0,2,0,0,8
2189,"[-2, 14, 7, -2]","[14, 19, 10]",9,"[2, -4, -4, -9, -14, 16, 18, 18, -23]",2324,"[{'D→nu_R+2fi': [-4, 2]}, {'D→nu_R+fi+fj': [-14, -4, 18]}, {'D→nu_R+2fi': [18, -9]}]",1,2,0,0,14


[3, 6, -12, -12, -12, 13, 14] →  Three massles states

In [45]:
dsf['3→2*x:0'].sort_values(['n'])[:2]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
0,"[-2, -6, 12]","[4, -5]",7,"[3, 6, -12, -12, -12, 13, 14]",120,"[{'D→nu_R+2fi': [-12, 6]}]",0,1,0,0
1,"[5, -4, 1]","[5, 15, 12]",8,"[5, -10, -10, -10, 13, 13, 22, -23]",18,"[{'D→nu_R+2fi': [-10, 5]}]",0,1,0,0


In [46]:
dsf['4→2*x:0'].sort_values(['n'])[:2]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
0,"[-8, 4, -2, 13]","[-3, 1, -5]",9,"[3, -6, -6, -6, -6, 12, 13, 14, -18]",750,"[{'D→nu_R+2fi': [-6, 3]}]",0,1,0,0


In [47]:
dsf['4→2*x:0'].sort_values(['n'])[:2]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
0,"[-8, 4, -2, 13]","[-3, 1, -5]",9,"[3, -6, -6, -6, -6, 12, 13, 14, -18]",750,"[{'D→nu_R+2fi': [-6, 3]}]",0,1,0,0


In [48]:
[c for c in dsf.keys() if c.find(':')>-1 and not c.find(':0')>-1]

['x→x→x:3',
 'nu_R→2*x:3',
 'x→x→x:4',
 'nu_R→2*x:4',
 '2→x→x:3',
 '3→x→x:3',
 '2→2→x:3',
 'nu_R→2*2:3']

In [49]:
dsf['x→x→x:3'].sort_values(['n','XDirac'])[:1]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
0,"[6, -6, 6]","[9, 0, -9]",8,"[13, -18, -18, -18, 22, 23, 23, -27]",729,"[{'X→nu_R+fi+fj': [-27, 22, 23]}]",0,0,1,0


In [50]:
dsf['2→2→x:3']

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,2→3sum
109,"[-6, -4, -5, -4]","[-4, -5, 9]",9,"[4, 4, 4, -5, -9, -10, -10, 11, 11]",91,"[{'X→nu_R+fi+fj': [-10, -5, 11]}]",0,0,1,0,5
116,"[1, -3, -4, -5]","[3, -1, 0]",9,"[2, 2, 2, -3, -3, 4, -5, -5, 6]",4,"[{'X→nu_R+fi+fj': [-5, -3, 6]}]",0,0,1,0,-6
225,"[-5, 0, 2, -10]","[15, 17, 15]",9,"[3, 10, 10, -12, -15, -15, -15, 17, 17]",470,"[{'X→nu_R+fi+fj': [-12, 10, 17]}]",0,0,1,0,12
232,"[6, -6, -4, 12]","[4, -10, 9]",9,"[4, 4, 6, 6, -7, -7, -7, -12, 13]",732,"[{'X→nu_R+fi+fj': [-12, 6, 13]}]",0,0,1,0,3


In [51]:
[c for c in dsf.keys() if c.find('*')>-1]

['2→2*x:0',
 '3→2*x:0',
 'nu_R→2*x:3',
 '4→2*x:0',
 'nu_R→2*x:4',
 '2→2*2:0',
 '3→2*2:0',
 '3→2*3:0',
 '4→2*3:0',
 'nu_R→2*2:3']

In [52]:
dsf['nu_R→2*x:3'].sort_values(['n','XDirac'])

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
0,"[-6, -10, -3, 12]","[18, 2, 9]",9,"[3, -8, 12, 12, 12, -18, -18, -19, 24]",853,"[{'X→nu_R+2fi': [-18, 3]}]",0,0,0,1
1,"[-1, -3, 7, 13]","[8, 9, 4]",9,"[5, 5, -11, 13, -15, -15, -15, 16, 17]",60,"[{'X→nu_R+2fi': [5, 5]}]",0,0,0,1
2,"[3, -7, -5, 12]","[-7, 13, 17]",9,"[1, 1, 3, -7, -7, -7, 19, 24, -27]",406,"[{'X→nu_R+2fi': [1, 3]}]",0,0,0,1
3,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1
4,"[2, -8, 7, 9]","[9, -9, -8]",9,"[1, -7, 9, 9, 9, -11, -11, -16, 17]",320,"[{'X→nu_R+2fi': [-11, 1]}]",0,0,0,1
5,"[-10, -8, 4, 9]","[8, 6, -2]",9,"[1, 1, 1, 2, -6, -9, 11, 11, -12]",20,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'X→nu_R+2fi': [11, -6]}]",1,0,0,1
6,"[-3, -9, 3, 5]","[12, 2, 7]",9,"[1, 1, -3, 5, 5, 5, -17, -21, 24]",150,"[{'X→nu_R+2fi': [1, -3]}]",0,0,0,1
7,"[-1, 13, 15, 13]","[8, 7, 0]",9,"[1, 13, 13, -15, -15, -15, -16, 17, 17]",620,"[{'X→nu_R+2fi': [13, 1]}]",0,0,0,1
8,"[-9, -4, 0, 4]","[2, -6, 9]",9,"[2, -6, -9, -9, -12, 21, 21, 21, -29]",264,"[{'D→nu_R+fi+fj': [-12, -9, 21]}, {'X→nu_R+2fi': [-9, -6]}]",1,0,0,1
9,"[-5, -9, -5, 5]","[-4, -5, -3]",9,"[5, 5, -8, 11, 13, -15, -15, -15, 19]",106,"[{'X→nu_R+2fi': [5, 5]}]",0,0,0,1


In [53]:
dsf['nu_R→2*2:3']

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
3,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1
7,"[-1, 13, 15, 13]","[8, 7, 0]",9,"[1, 13, 13, -15, -15, -15, -16, 17, 17]",620,"[{'X→nu_R+2fi': [13, 1]}]",0,0,0,1


All the published solutions already found

In [117]:
df=Table.read('table.tex').to_pandas()
df=pd.DataFrame( {'slt':[ list([ll for ll in l if ll!=0]) for l in df.values]} ).astype(str)
ds['slt']=ds['solution'].astype(str)

In [118]:
df.merge(ds,on='slt',how='left')

,slt,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor
0,"[2, 4, -7, -9, 10]","[-1, -2]",[2],5,"[2, 4, -7, -9, 10]",1,[],0,0,0,0
1,"[1, 5, -7, -8, 9]","[1, 4]",[-1],5,"[1, 5, -7, -8, 9]",2,[],0,0,0,0
2,"[1, 14, -17, -18, 20]","[10, 12]",[-5],5,"[1, 14, -17, -18, 20]",300,[],0,0,0,0
3,"[5, 6, -12, -21, 22]","[9, 4]",[6],5,"[5, 6, -12, -21, 22]",27,"[{'D→nu_R+2fi': [-12, 6]}]",0,1,0,0
4,"[7, 8, -18, -22, 25]","[3, 5]",[-3],5,"[7, 8, -18, -22, 25]",9,[],0,0,0,0
5,"[7, 9, -20, -22, 26]","[-5, 1]",[-3],5,"[7, 9, -20, -22, 26]",15,[],0,0,0,0
6,"[4, 9, -14, -25, 26]","[1, -4]",[-2],5,"[4, 9, -14, -25, 26]",1,[],0,0,0,0
7,"[7, 8, -17, -25, 27]","[-5, 5]",[-4],5,"[7, 8, -17, -25, 27]",50,[],0,0,0,0
8,"[5, 8, -14, -26, 27]","[2, 6]",[-10],5,"[5, 8, -14, -26, 27]",48,[],0,0,0,0
9,"[2, 18, -23, -25, 28]","[1, -1]",[-5],5,"[2, 18, -23, -25, 28]",1,[],0,0,0,0


In [75]:
 kk.columns

Index(['l', 'k', 'gcd', 'n', 'solution', 'DD', 'DM', 'XD', 'XM'], dtype='object')

{'DD': <function __main__.f>,
 'DM': <function __main__.f>,
 'XD': <function __main__.f>,
 'XM': <function __main__.f>,
 'gcd': <function __main__.f>,
 'k': <function __main__.f>,
 'l': <function __main__.f>,
 'n': <function __main__.f>,
 'solution': <function __main__.f>}

In [69]:
d

{'DD': <function __main__.f>,
 'DM': <function __main__.f>,
 'XD': <function __main__.f>,
 'XM': <function __main__.f>,
 'gcd': <function __main__.f>,
 'k': <function __main__.f>,
 'l': <function __main__.f>,
 'n': <function __main__.f>,
 'solution': <function __main__.f>}

In [71]:
{'l':f,'k':f}

{'k': <function __main__.f>, 'l': <function __main__.f>}

In [119]:
ds[ds['XMajor']>0]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,slt
7991,"[-6, -10, -3, 12]","[18, 2, 9]",9,"[3, -8, 12, 12, 12, -18, -18, -19, 24]",853,"[{'X→nu_R+2fi': [-18, 3]}]",0,0,0,1,"[3, -8, 12, 12, 12, -18, -18, -19, 24]"
8636,"[-1, -3, 7, 13]","[8, 9, 4]",9,"[5, 5, -11, 13, -15, -15, -15, 16, 17]",60,"[{'X→nu_R+2fi': [5, 5]}]",0,0,0,1,"[5, 5, -11, 13, -15, -15, -15, 16, 17]"
9461,"[3, -7, -5, 12]","[-7, 13, 17]",9,"[1, 1, 3, -7, -7, -7, 19, 24, -27]",406,"[{'X→nu_R+2fi': [1, 3]}]",0,0,0,1,"[1, 1, 3, -7, -7, -7, 19, 24, -27]"
10531,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1,"[1, 1, -4, -5, 9, 9, 9, -10, -10]"
11419,"[2, -8, 7, 9]","[9, -9, -8]",9,"[1, -7, 9, 9, 9, -11, -11, -16, 17]",320,"[{'X→nu_R+2fi': [-11, 1]}]",0,0,0,1,"[1, -7, 9, 9, 9, -11, -11, -16, 17]"
12150,"[-10, -8, 4, 9]","[8, 6, -2]",9,"[1, 1, 1, 2, -6, -9, 11, 11, -12]",20,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'X→nu_R+2fi': [11, -6]}]",1,0,0,1,"[1, 1, 1, 2, -6, -9, 11, 11, -12]"
12825,"[-3, -9, 3, 5]","[12, 2, 7]",9,"[1, 1, -3, 5, 5, 5, -17, -21, 24]",150,"[{'X→nu_R+2fi': [1, -3]}]",0,0,0,1,"[1, 1, -3, 5, 5, 5, -17, -21, 24]"
14999,"[-1, 13, 15, 13]","[8, 7, 0]",9,"[1, 13, 13, -15, -15, -15, -16, 17, 17]",620,"[{'X→nu_R+2fi': [13, 1]}]",0,0,0,1,"[1, 13, 13, -15, -15, -15, -16, 17, 17]"
16258,"[-9, -4, 0, 4]","[2, -6, 9]",9,"[2, -6, -9, -9, -12, 21, 21, 21, -29]",264,"[{'D→nu_R+fi+fj': [-12, -9, 21]}, {'X→nu_R+2fi': [-9, -6]}]",1,0,0,1,"[2, -6, -9, -9, -12, 21, 21, 21, -29]"
17360,"[-5, -9, -5, 5]","[-4, -5, -3]",9,"[5, 5, -8, 11, 13, -15, -15, -15, 19]",106,"[{'X→nu_R+2fi': [5, 5]}]",0,0,0,1,"[5, 5, -8, 11, 13, -15, -15, -15, 19]"


In [63]:
anomaly.free([5,0,-5],[10,-5,5])

array([ 1250, -3125, -3125, -3125,  4375,  5000,  5000, -6250])

In [64]:
anomaly.free.gcd

625

In [65]:
anomaly.free.simplified

array([  2,  -5,  -5,  -5,   7,   8,   8, -10])

In [125]:
ds[ds['solution'].apply(lambda l: np.abs(l).max())<=23].shape

(5100, 10)